In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pytorch_lightning as pl
from torch_geometric.loader import DataLoader
from torch.utils.data import DataLoader as DL
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import dense_to_sparse
import pandas as pd


In [83]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [84]:
## Loading node embedding data
node_embeddings = torch.load('./Output/node_embeddings_initial.pt')
node_embeddings = node_embeddings.to(device)

## Loading adjacency matrix
adj = torch.load('./Output/sub_adjacency_matrix.pt')
adj = adj.to(device)

/tmp/ipykernel_23694/2962403744.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  node_embeddings = torch.load('./Output/node_embeddings_initial.pt')
/tmp/ipykernel_23694/

In [85]:
# Define the F3 Classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_classes, num_gcn_layers=2):
        super(Classifier, self).__init__()
        
        # Graph Convolution Layers
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(GCNConv(input_dim, hidden_dim))  # First layer
        for _ in range(num_gcn_layers - 1):
            self.gcn_layers.append(GCNConv(hidden_dim, hidden_dim))  # Hidden layers
        
        # Fully Connected Layers
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, output_dim),
            nn.ReLU(),
            nn.Linear(output_dim, num_classes)
        )
        
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, h, adj):
        # Convert adjacency matrix to sparse format
        edge_index, edge_weight = dense_to_sparse(adj)
        
        # Apply GCN layers
        for gcn_layer in self.gcn_layers:
            h = gcn_layer(h, edge_index, edge_weight)
            h = torch.relu(h)
            h = self.dropout(h)
        
        # Fully Connected Layers
        logits = self.fc(h)
        return logits


In [86]:
# Early Stopping Parameters
patience = 50  # Number of epochs to wait for improvement
best_loss = float('inf')  # Initialize the best loss to infinity
patience_counter = 0  # Counter to track patience

# Training Loop with Early Stopping
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(node_embeddings, adj)  # Use loaded embeddings and adjacency matrix
    
    # Filter predictions for labeled nodes
    filtered_predictions = output[labeled_node_indices]
    
    # Compute loss
    loss = criterion(filtered_predictions, labels[:len(labeled_node_indices)])
    loss.backward()
    optimizer.step()

    # Check for validation loss improvement (can be training loss if no validation set)
    if loss.item() < best_loss:
        best_loss = loss.item()  # Update best loss
        patience_counter = 0  # Reset patience counter
        torch.save(model.state_dict(), "best_model.pth")  # Save the best model
    else:
        patience_counter += 1  # Increment patience counter

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Early stopping condition
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}. Best Loss: {best_loss:.4f}")
        break

# Load the best model after training
model.load_state_dict(torch.load("best_model.pth"))
print("Best model loaded for evaluation.")


Epoch [10/1000], Loss: 0.0000
Epoch [20/1000], Loss: 0.0000
Epoch [30/1000], Loss: 0.0000
Epoch [40/1000], Loss: 0.0000
Epoch [50/1000], Loss: 0.0000
Early stopping at epoch 52. Best Loss: 0.0000
Best model loaded for evaluation.


/tmp/ipykernel_23694/2770782362.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


In [87]:
# Evaluate the model
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculations for inference
    # Forward pass to get output logits
    output = model(node_embeddings, adj)
    
    # Predicted classes (index of the max probability for each node)
    predicted_classes = torch.argmax(output, dim=1)
    
    # Class probabilities (optional, for better insight)
    probabilities = torch.nn.functional.softmax(output, dim=1)

# Print results
print("\nFinal Predicted Classes for All Nodes:")
print(predicted_classes)


print("\nFinal Predicted Classes for All Nodes:")
print(predicted_classes.tolist())  # Converts the tensor to a Python list

print("\nClass Probabilities for All Nodes:")
print(probabilities)



Final Predicted Classes for All Nodes:
tensor([0, 1, 2,  ..., 0, 0, 0], device='cuda:0')

Final Predicted Classes for All Nodes:
[0, 1, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 2, 2, 2, 1, 0, 0, 0, 1, 0, 2, 0, 2, 1, 0, 0, 0, 2, 0, 2, 1, 0, 1, 2, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0, 1, 2, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 1, 2, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 1, 2, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 1, 0, 0, 0, 2, 0, 1, 2, 0, 0, 1, 2, 2, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 1, 0,